# Projet maison n° 3

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [1]:
# imports
import pandas as pd
import numpy as np
import os 

In [2]:
# Chargement des noms des fichiers .txt que l'on stock dans une liste 
txt_files = []
file_name = os.listdir('names/')

for f in file_name:
    if f.endswith('.txt'):
        txt_files.append(f)

In [3]:
def df_names_us():
    
    list_dfs = []
 
    for i, txt in enumerate(txt_files, start = 1880):
            df = pd.read_csv('names/'+txt, delimiter=',', names=['name', 'gender', 'births'], header = None)
            df['year'] = i
            df.sort_values(['gender', 'births'], ascending=[True, False])
            list_dfs.append(df)
    
    # Index dans le bon ordre 
    df = pd.concat(list_dfs)[["year", "name", "gender", "births"]]
    
    # Passage du type Int64Index à RangeIndex 
    df.reset_index(inplace=True, drop = True)
    
    return df

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [4]:
def df_names_fr():
    
    df_pf = pd.read_csv('nat2021.csv', delimiter=";")
    df_pf = df_pf.rename(columns = {'sexe':'gender', 'preusuel': 'name', 'annais' : 'year', 'nombre' : 'births'}) 
    df_pf = df_pf[['year', 'name', 'gender', 'births']]
    df_pf = df_pf[(df_pf['year'] != 'XXXX') & (df_pf['name'] != '_PRENOMS_RARES')]
    df_pf.dropna(inplace = True)
    df_pf['gender'] = df_pf['gender'].map({1 : 'M', 2: 'F'})
    df_pf = df_pf.astype({'year' : int}) 
    
    df_pf = df_pf[df_pf['name'].str.len() >= 2]
    df_pf['name'] = df_pf['name'].str.title()
    df_pf = df_pf.sort_values(['year','gender','births', 'name'], ascending=[True, True, False, True])
    df_pf.reset_index(inplace = True, drop = True)

    return df_pf

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [5]:
 def df_taux_change(devises):

    df_tc = pd.read_csv('Webstat_Export.csv', delimiter=';')
    df_tc = df_tc.drop(df_tc.index[0:5])
    df_tc = df_tc.rename(columns= {'Titre :' : 'Dates'})
    df_tc['Dates'] = pd.to_datetime(df_tc['Dates'], infer_datetime_format=True)
    
    # Méthode chaining 
    df_tc = (
        df_tc
        .set_index('Dates')
        .sort_index(ascending = True)
        .rename(columns= {c : c[c.find('(') + 1 : c.find(')')] for c in df_tc.columns})
        .drop(columns='100 paise')
        .apply(lambda x: x.str.replace(',', '.'))
        .apply(pd.to_numeric, args=('coerce',))
        .dropna()
    )
    
    return df_tc[devises]

### Tests 

## 

In [9]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        
        # --------------- IMPORTANT -------------
        self.assertEqual(len(df), 648329)  # 648330 à 648329 car il y avait une valeur nulle parmis les 648330 
                                           # donc la suite des tests n'etaient pas réalisable, en particulier le dernier test
                                           # Je me permets donc de supposer 
                                           # que vous n'aviez pas vu le NaN présent dans la colonne 'name' parmis les 648330
                                           
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [10]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [11]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 5.269s

OK
